In [1]:
import os, sys
root_dir = os.path.dirname(os.getcwd())
sys.path.insert(0, root_dir)

In [2]:
from models.basemodel import BaseModel
import numpy as np
from mothernet.prediction.tabpfn import TabPFNClassifier
import torch


/data/yzeng58/anaconda3/envs/torch23/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
model_tabpfn = TabPFNClassifier(
    device='cuda', 
    N_ensemble_configurations=32, 
    model_string = 'prior_diff_real_checkpoint_n_0',
    epoch = '100',
)


In [7]:
model_tabflex = TabPFNClassifier(
    device='cuda', 
    model_string = f'tabpfn_modelnaive_linear_attention_validdatanew_11_24_2024_22_30_15',
    N_ensemble_configurations=1,
    epoch = 'on_exit', 
)

In [8]:

X = np.random.randn(100, 100)
y = np.random.randint(0, 10, (100,))
model_tabpfn.fit(X, y)
model_tabflex.fit(X, y)

for name, param in model_tabpfn.model.named_parameters():
    print(name)


Number of parameters in backbone:  25233408
y_encoder.weight
y_encoder.bias
transformer_encoder.layers.0.self_attn.in_proj_weight
transformer_encoder.layers.0.self_attn.in_proj_bias
transformer_encoder.layers.0.self_attn.out_proj.weight
transformer_encoder.layers.0.self_attn.out_proj.bias
transformer_encoder.layers.0.linear1.weight
transformer_encoder.layers.0.linear1.bias
transformer_encoder.layers.0.linear2.weight
transformer_encoder.layers.0.linear2.bias
transformer_encoder.layers.0.norm1.weight
transformer_encoder.layers.0.norm1.bias
transformer_encoder.layers.0.norm2.weight
transformer_encoder.layers.0.norm2.bias
transformer_encoder.layers.1.self_attn.in_proj_weight
transformer_encoder.layers.1.self_attn.in_proj_bias
transformer_encoder.layers.1.self_attn.out_proj.weight
transformer_encoder.layers.1.self_attn.out_proj.bias
transformer_encoder.layers.1.linear1.weight
transformer_encoder.layers.1.linear1.bias
transformer_encoder.layers.1.linear2.weight
transformer_encoder.layers.1.l

In [11]:
# Copy parameter values from model_tabpfn to model_tabflex
with torch.no_grad():
    for (name_target, target_param), (name_source, source_param) in zip(model_tabflex.model.named_parameters(), model_tabpfn.model.named_parameters()):
        try:
            target_param.copy_(source_param)
        except:
            print(f"Failed to copy parameter {name_target} from {name_source}")
        
model = model_tabflex



Failed to copy parameter ssm.layers.0.attention.query_projection.weight from transformer_encoder.layers.0.self_attn.in_proj_weight
Failed to copy parameter ssm.layers.0.attention.query_projection.bias from transformer_encoder.layers.0.self_attn.in_proj_bias
Failed to copy parameter ssm.layers.0.attention.value_projection.weight from transformer_encoder.layers.0.linear1.weight
Failed to copy parameter ssm.layers.0.attention.value_projection.bias from transformer_encoder.layers.0.linear1.bias
Failed to copy parameter ssm.layers.0.attention.out_projection.weight from transformer_encoder.layers.0.linear2.weight
Failed to copy parameter ssm.layers.0.linear1.bias from transformer_encoder.layers.0.norm1.bias
Failed to copy parameter ssm.layers.0.linear2.weight from transformer_encoder.layers.0.norm2.weight
Failed to copy parameter ssm.layers.0.norm1.weight from transformer_encoder.layers.1.self_attn.in_proj_weight
Failed to copy parameter ssm.layers.0.norm1.bias from transformer_encoder.layer

In [4]:

X = np.random.randn(100, 100)
y = np.random.randint(0, 10, (100,))
model.fit(X, y)


Number of parameters in backbone:  25233408


TabPFNClassifier(N_ensemble_configurations=32, device='cuda', epoch='100',
                 model=TabPFN(
  (y_encoder): Linear(in_features=1, out_features=512, bias=True)
  (transformer_encoder): TransformerEncoderDiffInit(
    (layers): ModuleList(
      (0-11): 12 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=512, out_features...
        (norm2): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.0, inplace=False)
        (dropout2): Dropout(p=0.0, inplace=False)
      )
    )
  )
  (encoder): Linear(in_features=100, out_features=512, bias=True)
  (decoder): Sequential(
    (0): Linear(in_features=512, out_features=1024, bias=True)
    (1): GELU(approximate='none')
    (2): Linear(in_features=1024, out_features=10, bias=True)
  )
),
                 model_string='prior_diff_real_checkpoint_n_0')

In [5]:
for name, param in model.model.named_parameters():
    if 'attn' in name:
        print(name)


transformer_encoder.layers.0.self_attn.in_proj_weight
transformer_encoder.layers.0.self_attn.in_proj_bias
transformer_encoder.layers.0.self_attn.out_proj.weight
transformer_encoder.layers.0.self_attn.out_proj.bias
transformer_encoder.layers.1.self_attn.in_proj_weight
transformer_encoder.layers.1.self_attn.in_proj_bias
transformer_encoder.layers.1.self_attn.out_proj.weight
transformer_encoder.layers.1.self_attn.out_proj.bias
transformer_encoder.layers.2.self_attn.in_proj_weight
transformer_encoder.layers.2.self_attn.in_proj_bias
transformer_encoder.layers.2.self_attn.out_proj.weight
transformer_encoder.layers.2.self_attn.out_proj.bias
transformer_encoder.layers.3.self_attn.in_proj_weight
transformer_encoder.layers.3.self_attn.in_proj_bias
transformer_encoder.layers.3.self_attn.out_proj.weight
transformer_encoder.layers.3.self_attn.out_proj.bias
transformer_encoder.layers.4.self_attn.in_proj_weight
transformer_encoder.layers.4.self_attn.in_proj_bias
transformer_encoder.layers.4.self_attn

In [7]:
# Check the path of the forward function
print(model.model.transformer_encoder.layers[0].self_attn)

MultiheadAttention(
  (out_proj): NonDynamicallyQuantizableLinear(in_features=512, out_features=512, bias=True)
)


In [8]:
# Check which file defines the MultiheadAttention class
print(model.model.transformer_encoder.layers[0].self_attn.__class__.__module__)

mothernet.models.flash_transformer


In [16]:
torch._native_multi_head_attention

AttributeError: 'builtin_function_or_method' object has no attribute '__file__'

In [11]:
model.model.transformer_encoder.layers[0].self_attn.forward

<bound method MultiheadAttention.forward of MultiheadAttention(
  (out_proj): NonDynamicallyQuantizableLinear(in_features=512, out_features=512, bias=True)
)>

In [ ]:
# Change all attention layers to use linear attention instead of softmax
for name, module in model.model.named_modules():
    if isinstance(module, torch.nn.MultiheadAttention):
        # Set attn_name to flash_linear_attention to use linear attention
        module.attn_name = 'flash_linear_attention'
        
        # Create new attention module with linear attention
        new_attn = MultiheadAttention(
            module.embed_dim,
            module.num_heads,
            dropout=module.dropout,
            batch_first=True,
            attn_name='flash_linear_attention',
            feature_map='identity',
            norm_output=False
        ).to(module.in_proj_weight.device)
        
        # Copy weights from old attention
        new_attn.in_proj_weight = module.in_proj_weight
        new_attn.in_proj_bias = module.in_proj_bias
        new_attn.out_proj.weight = module.out_proj.weight
        new_attn.out_proj.bias = module.out_proj.bias
        
        # Replace old attention with new linear attention
        module.self_attn = new_attn
